In [1]:
import numpy as np
from hottbox.core import Tensor
import yfinance as yf
import alphalens
import pandas as pd
from hottbox.pdtools import tensor_to_pd, pd_to_tensor
from hottbox.utils.generation import residual_tensor
from hottbox.algorithms.decomposition import TTSVD, HOSVD, HOOI, CPD
from hottbox.metrics import residual_rel_error, mse, rmse, mape
from hottbox.rank import rank_estimation as re
from matplotlib import pyplot as plt

In [2]:
pan = yf.download("BAC JPM GS C MS HSBC BNPQY BCS LYG UBS CS WFC RY SCHW TD HDB USB PNC TFC BNS", start="2011-05-21", end="2019-05-21")
pan

[*********************100%***********************]  20 of 20 completed


Adj Close                                                         \
                  BAC        BCS      BNPQY        BNS          C         CS   
Date                                                                           
2011-05-20  10.121597  12.614285  25.608244  38.149796  35.830624  29.171848   
2011-05-23   9.981748  12.280837  25.040247  37.712025  35.079422  28.720551   
2011-05-24  10.016708  12.280837  25.074883  37.775471  35.385147  28.727602   
2011-05-25   9.946785  12.585903  25.594391  37.997532  35.236618  29.475065   
2011-05-26  10.016708  12.749084  25.802195  38.454338  35.140503  30.067381   
...               ...        ...        ...        ...        ...        ...   
2019-05-14  27.256250   7.606107  23.181927  47.471714  60.872921  11.398154   
2019-05-15  26.941975   7.653001  23.081570  47.409000  60.509144  11.398154   
2019-05-16  27.227682   7.690516  23.355267  47.588177  61.544521  11.455817   
2019-05-17  27.046730   7.521700  22.962971  47.552345  60.695690  11.330880   
2019-05-20  27.046730   7.446670  22.771385  47.632969  60.593090  11.273216   

                                                          ...   Volume  \
                    GS        HDB       HSBC         JPM  ...      LYG   
Date                                                      ...            
2011-05-20  115.233727  14.786564  30.708990   32.632988  ...  3168800   
2011-05-23  115.959290  14.555243  30.131813   32.194149  ...  2852600   
2011-05-24  116.386108  14.610723  30.054462   32.035255  ...  3323800   
2011-05-25  116.300774  14.512930  30.280577   31.982307  ...  2784400   
2011-05-26  116.317818  15.113795  30.512636   32.118481  ...  2607600   
...                ...        ...        ...         ...  ...      ...   
2019-05-14  187.180862  57.678493  38.378349  103.506302  ...  4688200   
2019-05-15  187.295273  57.042976  38.751659  103.112236  ...  3021000   
2019-05-16  189.908264  58.100517  38.908287  104.435150  ...  2849800   
2019-05-17  188.277512  58.040943  38.337059  103.928497  ...  3670400   
2019-05-20  188.096375  61.506516  38.161999  104.472672  ...  3814900   

                                                                             \
                  MS      PNC       RY      SCHW       TD      TFC      UBS   
Date                                                                          
2011-05-20  12566000  3038800   815300   8851700  1872600  7646300  3114000   
2011-05-23  10709200  2820600   374200   7008300  1487000  3654500  1960600   
2011-05-24  10429200  2598300   523500   7179900  1763800  4077500  3181900   
2011-05-25  10188000  2798400   522400   5939400  1471400  7638400  2525300   
2011-05-26   9428700  2666600   868000   7394900  1890800  6224300  2302900   
...              ...      ...      ...       ...      ...      ...      ...   
2019-05-14  17302200  1088500   556400   5098200   969000  3726000  3149100   
2019-05-15  10703100  1609200   526200  10343600  1006500  4005700  3025200   
2019-05-16   9305000  1508900   635300   8458500  1076100  3638900  6006900   
2019-05-17   8411700  1351600   829700   8971000   833800  7093700  2335400   
2019-05-20   8951600  1153200  1138700   7055000  2028300  4101600  3034400   

                                
                 USB       WFC  
Date                            
2011-05-20  10140600  29293400  
2011-05-23  10139300  28094700  
2011-05-24   7976400  21103400  
2011-05-25   7837400  20648700  
2011-05-26   8998300  18407800  
...              ...       ...  
2019-05-14   6210900  21267600  
2019-05-15   4071000  20123300  
2019-05-16   4460200  20764100  
2019-05-17   5278400  22273500  
2019-05-20   4573000  18854000  

[2012 rows x 120 columns]

In [3]:
HML = pan["High"]-pan["Low"]
HML = HML.stack()
HML.index = HML.index.set_names(['date', 'asset'])
HML = HML.unstack()
HML = HML.stack()
HML

date        asset
2011-05-20  BAC      0.250000
            BCS      0.304150
            BNPQY    0.849998
            BNS      0.690002
            C        0.680000
                       ...   
2019-05-20  TD       0.639999
            TFC      0.739998
            UBS      0.145000
            USB      0.880001
            WFC      0.610001
Length: 40240, dtype: float64

In [4]:
#log returns

returns = pan["Adj Close"].apply(np.log).diff(1)
returns = returns.stack()
returns.index = returns.index.set_names(['date', 'asset'])
returns = returns.unstack()
# returns.index = returns.index.tz_localize('UTC')
returns = returns.stack()
returns

date        asset
2011-05-23  BAC     -0.013913
            BCS     -0.026790
            BNPQY   -0.022430
            BNS     -0.011541
            C       -0.021188
                       ...   
2019-05-20  TD       0.006495
            TFC      0.009562
            UBS      0.003339
            USB      0.003098
            WFC     -0.005486
Length: 40220, dtype: float64

# SIMFIN

In [5]:
%matplotlib inline
import pandas as pd
import seaborn as sns

# Import the main functionality from the SimFin Python API.
import simfin as sf

# Import names used for easy access to SimFin's data-columns.
from simfin.names import *

In [6]:
# Version of the SimFin Python API.
sf.__version__

'0.8.1'

In [7]:
# SimFin data-directory.
sf.set_data_dir('~/simfin_data/')

In [8]:
# SimFin load API key or use free data.
sf.load_api_key(path='~/simfin_api_key.txt', default_key='free')

In [9]:
# Seaborn set plotting style.
sns.set_style("whitegrid")

In [72]:
# We are interested in the US stock-market.
market = 'us'

# List of tickers we want. If None then all tickers are used.
tickers = sorted(["GOOG","AAPL", "AMD", "TSLA", "NFLX", "TXN", "AMAT", "AMZN", "INTC", "NVDA", "ADBE", "CSCO", "V", "IBM", "QCOM", "MSFT", "XLNX", "HPQ", "VZ", "T"])
# tickers = sorted(["BAC", "JPM", "GS", "C", "MS", "HSBC", "BNPQY", "BCS", "LYG", "UBS", "CS", "WFC", "RY", "SCHW", "TD", "HDB", "USB", "PNC", "TFC", "BNS"])

# Add this date-offset to the fundamental data such as
# Income Statements etc., because the REPORT_DATE is not
# when it was actually made available to the public,
# which can be 1, 2 or even 3 months after the Report Date.
offset = pd.DateOffset(days=60)

# Refresh the fundamental datasets (Income Statements etc.)
# every 30 days.
refresh_days = 30

# Refresh the dataset with shareprices every 10 days.
refresh_days_shareprices = 10


In [73]:
%%time
hub = sf.StockHub(market=market, tickers=tickers, offset=offset,
                  refresh_days=refresh_days,
                  refresh_days_shareprices=refresh_days_shareprices,
                 )

CPU times: user 54 µs, sys: 64 µs, total: 118 µs
Wall time: 156 µs


In [74]:
%%time
df_val_signals_daily = hub.val_signals(variant='daily')

Dataset "us-shareprices-daily" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-income-ttm" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-balance-ttm" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-cashflow-ttm" on disk (0 days old).
- Loading from disk ... Done!
Cache-file 'val_signals-4c3c6a0f.pickle' on disk (0 days old).
- Loading from disk ... Done!
CPU times: user 39.1 s, sys: 5.15 s, total: 44.2 s
Wall time: 49.3 s


In [59]:
df_factor = pd.DataFrame()
for i in tickers:
    df_factor[i] = df_val_signals_daily.loc[i]['2011-05-20':'2019-05-20']["Price to Book Value"]
df_factor.fillna(0,inplace=True)
df_factor

,AAPL,ADBE,AMAT,AMD,AMZN,CSCO,GOOG,HPQ,IBM,INTC,MSFT,NFLX,NVDA,QCOM,T,TSLA,TXN,V,VZ,XLNX
Date,,,,,,,,,,,,,,,,,,,,
2011-05-20,5.689950,3.387547,0.000000,6.237374,13.197028,2.073305,0.000000,2.010652,9.453495,2.675726,4.424956,46.021893,3.347296,4.157249,1.661261,0.000000,4.067740,4.746807,1.210948,4.436839
2011-05-23,5.675690,3.303122,0.000000,6.143307,13.035594,2.050729,0.000000,2.001152,9.347938,2.634242,4.367137,46.375606,3.343595,4.084797,1.648531,0.000000,3.980574,4.666723,1.201170,4.320244
2011-05-24,5.637662,3.275300,0.000000,6.070948,12.839615,2.040694,0.000000,2.010652,9.332938,2.605433,4.363523,46.336304,3.356547,4.079726,1.652774,0.000000,3.960816,4.707978,1.202799,4.273109
2011-05-25,5.718471,3.274341,0.000000,6.099891,12.772517,2.030660,0.000000,2.012329,9.319604,2.613499,4.370750,48.563381,3.397255,4.128993,1.644288,0.000000,3.984061,4.778355,1.186501,4.281792
2011-05-26,5.685197,3.313675,0.000000,6.136071,12.954545,2.038186,0.000000,2.042505,9.287937,2.592757,4.457479,49.362509,3.482372,4.142034,1.650122,0.000000,3.988709,4.809904,1.193346,4.309080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-05-14,7.846893,13.674449,4.871482,22.960885,21.127006,5.970203,4.731782,-16.023707,7.215242,2.847849,10.560581,29.769314,10.840826,33.325666,1.091367,6.880068,12.002402,12.991365,4.270967,10.917490
2019-05-15,7.941734,13.923935,5.052899,23.179400,21.483272,6.018406,4.916629,-16.400220,7.274237,2.876220,10.669802,30.577265,10.676247,33.375966,1.096983,6.869702,12.139993,13.200576,4.290604,11.039995
2019-05-16,7.906793,14.262452,5.072380,23.540790,21.901421,6.418944,4.979005,-16.584099,7.354340,2.870546,10.916184,30.949371,10.717057,32.041082,1.109972,6.763081,12.021114,13.369243,4.333653,10.237437


In [27]:
df1 = df_factor.stack()
df1.index = df1.index.set_names(['date', 'asset'])
df1 = df1.unstack()
df1 = df1.stack()
df1.fillna(0)
df1

date        asset
2011-05-20  AAPL      5.689950
            ADBE      3.387547
            AMAT      0.000000
            AMD       6.237374
            AMZN     13.197028
                       ...    
2019-05-20  TSLA      6.081886
            TXN      11.527985
            V        13.255717
            VZ        4.456760
            XLNX      9.745485
Length: 40240, dtype: float64

In [18]:
# df_val_signals_daily.dropna(how='all')